In [1]:
import time
import requests
from bs4 import BeautifulSoup
import pymongo

In [2]:
mc = pymongo.MongoClient()  # Connect to the MongoDB server using default settings
db = mc['strain_reviews']  # Use (or create) a database called 'election_predictions'
docs = db['review']  # Use (or create) a collection called 'docs'

In [3]:
docs.count_documents(filter={})

0

In [4]:
docs.drop()

"""
#Here is how i want the scrape to work
LOS = [List of Strains]
    for s in LOS:
    url = "https://www.leafly.com/{}/reviews?page=".format(s)
    for i in range(1,100):
        d = get_reviews(url+str(i))
        st, rev = parse_docs(d)
        if len(st) > 0:
           append.(add to your dictionary)
        else:
            break"""

In [7]:
def table_check2(l):
    """This checks to see if a url is already in my database of reviews. 
    Only needs to be run in the get reviews function"""
    for url in l:
        if docs.find_one({'url':url}):
            None
        else:
            docs.insert_one({'url': url,
                 'html': requests.get(url).content
                 })

In [5]:
def get_reviews2(l):
    """Pass in a list of URL's and return them in a mongo db table as a dicitonary with 
    {'url', 'html'} and their corresponding values"""
    r = requests.get(l)
    html = (r.content)
    return html

In [8]:
def get_stars_list(d):
    stars = []
    for key, values in d.items():
        soup = BeautifulSoup(values, 'html.parser')
        tags = soup.select("div.div.stars")
        for t in tags:
            stars.append(t.attrs['style'])
    return stars

In [9]:
def star_int_conv(l):
    flat_stars= []
    star_num = []
    for s in l:
        star = int((s[6:].split(';')[0]).strip('px'))
        star_num.append((star/22))
        for star in star_num:
            for item in sublist:
                flat_stars.append(item)
    return flat_stars

In [10]:
def list_o_strains(i):
    LOS = []
    r = requests.get(i)
    soup2 = BeautifulSoup(r.content, 'html.parser')
    strains = soup2.find_all('a', class_="ga_Explore_Strain_Tile")
    for s in strains:
        LOS.append(s.attrs['href'])
    return LOS

In [6]:
def parse_docs2(d):
    """Parse the HTML docs that we have stored in a dictionary, return as a list.
    Also scrape and parse star rating for each review """
    strain_text= []
    star_num = []
    soup = BeautifulSoup(d, 'html.parser')
    revs = soup.find_all('p',class_='strain-review__text') 
    for r in revs:
        strain_text.append(r.text)
    tags = soup.select("div.div.stars")
    stars = []
    for t in tags:
        stars.append(t.attrs['style'])
    star_num.append(star_int_conv(stars))
    return star_num, strain_text

In [ ]:
def scraper_dummy(los):
    los_urls = []
    rev_urls = []
    for s in los:
        url = "https://www.leafly.com{}/reviews?page=".format(s)
        print(url)
        for i in range(1,100):
            rev_urls=url+str(i)
            d = get_reviews2(rev_urls)
            star, reviews = parse_docs2(d)
            print(len(star))
            if len(star) == 0:
                break
            #do something with (star, reviews)
            